In [1]:
import ee
from datetime import datetime

ee.Authenticate()
ee.Initialize()

Opening in existing browser session.

Successfully saved authorization token.


In [6]:
import pandas as pd

random_point_df = pd.read_csv('/media/kursat/TOSHIBA EXT7/projects/satellite/YENI/data/10k_random.csv')
coordinates = random_point_df[['lat', 'lon']].values.tolist()

start_date = '2022-01-01'
end_date = '2023-01-01'

In [7]:
dataset = ee.ImageCollection('COPERNICUS/S2')

# filter according to time interval
filtered_dataset = dataset.filterDate(ee.Date(start_date), ee.Date(end_date))

In [ ]:
import pandas as pd

# Assuming end_date is a string in the format 'YYYY-MM-DD'
end_date = ee.Date(end_date)

data_list = []
df_result_list = []

for i in range(len(coordinates)):
    print("Number of point: ", i)
    point = ee.Geometry.Point(coordinates[i])
    
    current_date = ee.Date(start_date)
    while current_date.format('YYYY-MM-dd').getInfo() <= end_date.format('YYYY-MM-dd').getInfo():
        current_image = filtered_dataset.filterBounds(point).first()
        # print("current_image")
        if current_image is not None:
            rgb_image = current_image.select(['B4', 'B3', 'B2'])
            
            # Use reduceRegion to get all values, then extract the specific band value
            pixel_intensity_dict = rgb_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=point, scale=10).getInfo()
            pixel_intensity = pixel_intensity_dict.get('B3')

            data_list.append({
                'date': current_date.format('YYYY-MM-dd').getInfo(),
                'pixel_intensity': pixel_intensity
                })
        
        # print(current_date)
        current_date = current_date.advance(1, 'day')
    
    df = pd.DataFrame(data_list)

    print(df)
    df_result_list.append(df)

In [4]:
df.to_csv('data/100points_pixelIntensitiesB3.csv')